# Bailey Pairs and Chain Discovery

A **Bailey pair** relative to $a$ is a pair of sequences $(\alpha_n, \beta_n)$ satisfying:

$$\beta_n = \sum_{r=0}^{n} \frac{\alpha_r}{(q;q)_{n-r}(aq;q)_{n+r}}$$

The **Bailey lemma** (also called **Bailey's transform**) shows that from one Bailey
pair, one can derive infinitely many others by iterating a transformation with
parameters $b$ and $c$. This iteration produces a **Bailey chain**.

Bailey pairs and chains are the engine behind many partition identities, including
the celebrated Rogers-Ramanujan identities.

## The Weak Bailey Lemma

The **weak Bailey lemma** states that if $(\alpha_n, \beta_n)$ is a Bailey pair
relative to $a$, then:

$$\sum_{n \geq 0} q^{n^2} a^n \beta_n = \frac{1}{(aq;q)_\infty} \sum_{n \geq 0} q^{n^2} a^n \alpha_n$$

Let's verify this with the **Rogers-Ramanujan** Bailey pair and $a = q$:

In [1]:
from q_kangaroo import QSession, bailey_weak_lemma

s = QSession()

# Verify the weak Bailey lemma with the Rogers-Ramanujan pair, a = q
lhs, rhs = bailey_weak_lemma(s, "rogers-ramanujan", 1, 1, 1, 10, 20)
print("LHS:", lhs)
print("RHS:", rhs)
print("LHS == RHS:", repr(lhs) == repr(rhs))

Both sides match, confirming the weak Bailey lemma for this pair.

## Applying the Bailey Lemma

The full **Bailey lemma** transforms a Bailey pair $(\alpha_n, \beta_n)$ relative
to $a$ into a new pair $(\alpha_n', \beta_n')$ using additional parameters $b$ and $c$.
This is the core step that enables Bailey chains.

The `bailey_apply_lemma` function applies one step of this transformation:

In [2]:
from q_kangaroo import bailey_apply_lemma

# Apply Bailey lemma to the unit pair with a=q^2, b=q/2, c=q/3
derived = bailey_apply_lemma(s, "unit", (1,1,2), (1,2,1), (1,3,1), 4, 15)
derived

{'name': "lemma(unit, b=QMonomial { coeff: QRat(1/2), power: 1 }, c=QMonomial { coeff: QRat(1/3), power: 1 })", 'pair_type': 'tabulated', 'num_terms': 5}

The result describes the derived Bailey pair: a tabulated pair with 5 terms,
obtained by applying the lemma to the unit pair.

## Building a Bailey Chain

A **Bailey chain** applies the lemma iteratively, producing a sequence of
derived pairs. Each step produces new partition identities.

The `bailey_chain` function builds a chain of specified depth:

In [3]:
from q_kangaroo import bailey_chain

# Build a depth-2 Bailey chain from the unit pair
chain = bailey_chain(s, "unit", (1,1,2), (1,2,1), (1,3,1), 2, 4, 15)

print(f"Chain length: {len(chain)}")
for step in chain:
    print(f"Step {step['index']}: {step['name']} ({step['pair_type']})")

The chain starts from the unit pair (step 0) and applies the lemma twice,
producing two derived tabulated pairs.

## Rogers-Ramanujan Identities from Bailey Pairs

The **Rogers-Ramanujan identities** are among the most beautiful results in partition theory:

$$\sum_{n=0}^{\infty} \frac{q^{n^2}}{(q;q)_n} = \prod_{n=0}^{\infty} \frac{1}{(1-q^{5n+1})(1-q^{5n+4})}$$

$$\sum_{n=0}^{\infty} \frac{q^{n^2+n}}{(q;q)_n} = \prod_{n=0}^{\infty} \frac{1}{(1-q^{5n+2})(1-q^{5n+3})}$$

These can be verified using the Bailey chain machinery. The Rogers-Ramanujan pair is a canonical pair in the database, and applying the weak Bailey lemma at different values of $a$ produces both identities.

With $a = 1$, the weak Bailey lemma gives the **first** Rogers-Ramanujan identity:

In [4]:
# First Rogers-Ramanujan identity: a = 1 (a_num=1, a_den=1, a_pow=0)
lhs_rr1, rhs_rr1 = bailey_weak_lemma(s, "rogers-ramanujan", 1, 1, 0, 10, 25)
print("First R-R identity (a=1):")
print("Sum side:    ", lhs_rr1)
print("Product side:", rhs_rr1)
print("Match:", repr(lhs_rr1) == repr(rhs_rr1))

In [5]:
# Second Rogers-Ramanujan identity: a = q (a_num=1, a_den=1, a_pow=1)
lhs_rr2, rhs_rr2 = bailey_weak_lemma(s, "rogers-ramanujan", 1, 1, 1, 10, 25)
print("Second R-R identity (a=q):")
print("Sum side:    ", lhs_rr2)
print("Product side:", rhs_rr2)
print("Match:", repr(lhs_rr2) == repr(rhs_rr2))

Both identities are simultaneously verified by the Bailey chain machinery. The coefficients $1, 1, 1, 1, 2, 2, 3, 3, 4, 5, 6, 7, 9, 10, 12, \ldots$ of the first identity count the number of partitions of $n$ into parts $\equiv \pm 1 \pmod{5}$, which also equals the number of partitions into parts differing by at least 2.

The second identity's coefficients $1, 0, 1, 1, 1, 1, 2, 2, 3, 3, 4, 4, 6, 6, 8, \ldots$ count partitions into parts $\equiv \pm 2 \pmod{5}$ (equivalently, partitions into parts $\geq 2$ differing by at least 2).

## The Unit Pair and the Weak Bailey Lemma

The **unit pair** is the simplest possible Bailey pair: $\alpha_0 = 1$, $\alpha_n = 0$ for $n > 0$. Despite its simplicity, the weak Bailey lemma applied to the unit pair produces a non-trivial identity:

$$\text{LHS} = \sum_{n \geq 0} q^{n^2} a^n \beta_n^{\text{unit}} \qquad \text{RHS} = \frac{1}{(aq;q)_\infty}$$

Since $\alpha_n = 0$ for $n > 0$, the RHS simplifies to $1/(aq;q)_\infty$.

In [6]:
# Unit pair with a = q
lhs_unit, rhs_unit = bailey_weak_lemma(s, "unit", 1, 1, 1, 10, 25)
print("Unit pair, weak lemma (a=q):")
print("LHS:", lhs_unit)
print("RHS:", rhs_unit)
print("Match:", repr(lhs_unit) == repr(rhs_unit))

## Automated Bailey Pair Discovery

Given two series, `bailey_discover` searches the database to determine whether
they are related by a Bailey pair:

In [7]:
from q_kangaroo import bailey_discover

# Verify that the two sides of the weak Bailey lemma are recognized
result = bailey_discover(s, lhs, rhs, (1,1,1), 2, 20)

print(f"Found: {result['found']}")
print(f"Verification: {result['verification']}")

The discovery engine confirms the two series are equal (direct equality),
which is expected since they are both sides of a verified Bailey lemma identity.

## Multi-Step Chains: Depth 3

The power of Bailey chains is that each step of the lemma produces a new pair, which can be fed back in. A depth-3 chain applies the lemma three times, producing pairs at increasingly higher "levels."

In [8]:
# Build a depth-3 chain from the unit pair
chain3 = bailey_chain(s, "unit", (1,1,2), (1,2,1), (1,3,1), 3, 4, 15)

print(f"Chain length: {len(chain3)}")
for step in chain3:
    print(f"Step {step['index']}: {step['name']} ({step['pair_type']})")

The chain has 4 entries (depth 3 = 3 applications of the lemma starting from step 0): Unit $\to$ Tabulated $\to$ Tabulated $\to$ Tabulated. Each step produces more complex pairs whose names reflect the nesting of lemma applications.

Andrews used chains of depth up to 5-6 to produce his multiple series generalizations of the Rogers-Ramanujan identities. Each additional depth level introduces a new summation variable, yielding $k$-fold sum identities.

## The q-Binomial Bailey Pair

The third canonical pair in the database is the **q-binomial pair**. It is related to q-binomial coefficient identities and produces different families of partition identities when fed through the chain.

Let us verify the weak Bailey lemma for this pair:

In [9]:
# Verify weak Bailey lemma for the q-binomial pair with a = q
lhs_qb, rhs_qb = bailey_weak_lemma(s, "q-binomial", 1, 1, 1, 8, 20)
print("q-binomial pair, weak lemma (a=q):")
print("LHS:", lhs_qb)
print("RHS:", rhs_qb)
print("Match:", repr(lhs_qb) == repr(rhs_qb))

The q-binomial pair produces a different generating function with slower initial growth ($0$ coefficient at $q^1$ and $q^2$). The coefficients $1, 0, 0, 1, 1, 1, 2, 3, 3, 5, 6, 8, 11, 14, 17, \ldots$ enumerate a different class of partition-like objects related to $q$-binomial coefficients.

## Automated Discovery with Different Inputs

The `bailey_discover` function searches the database of known pairs and chain constructions to determine if two given series are related by a Bailey pair transformation. Let us construct series from a known pair and see if discovery can identify the relationship.

In [10]:
# Construct LHS and RHS from the unit pair weak lemma
test_lhs, test_rhs = bailey_weak_lemma(s, "unit", 1, 1, 1, 8, 20)

# Now see if discovery can find this relationship
disc = bailey_discover(s, test_lhs, test_rhs, (1,1,1), 2, 20)
print(f"Found: {disc['found']}")
print(f"Verification: {disc['verification']}")

In [11]:
from q_kangaroo import partition_gf, theta3

# Try discovery with two series that are NOT Bailey-pair related
pgf = partition_gf(s, 20)
t3 = theta3(s, 20)
disc2 = bailey_discover(s, pgf, t3, (1,1,1), 2, 20)
print(f"Found: {disc2['found']}")

The discovery engine correctly identifies that the unit pair's weak lemma output matches itself (`direct equality`), while the partition generating function and $\theta_3$ are **not** related by any Bailey pair in the database. The partition generating function $\prod_{n=1}^{\infty} 1/(1-q^n)$ and $\theta_3(q) = \sum_{n=-\infty}^{\infty} q^{n^2}$ arise from fundamentally different structures.

The discovery algorithm proceeds in three stages:
1. **Trivial equality**: checks if LHS $=$ RHS directly
2. **Weak lemma matching**: tests each database pair
3. **Chain depth search**: iterates the Bailey lemma up to the specified depth

## Significance for Partition Identities

Bailey pairs and chains represent one of the most powerful tools in combinatorial number theory:

1. **Rogers-Ramanujan identities** (1894/1917): The first and most famous pair. The sum side counts partitions into parts differing by at least 2; the product side counts partitions into parts $\equiv \pm 1 \pmod{5}$. We verified both identities above using the weak Bailey lemma at $a = 1$ and $a = q$.

2. **Andrews' multiple series identities** (1984): By iterating Bailey chains to depth $k$, Andrews produced $k$-fold sum identities generalizing Rogers-Ramanujan. Our depth-3 chain above illustrates this mechanism, generating increasingly complex tabulated pairs.

3. **Slater's list** (1952): A systematic enumeration of 130 Rogers-Ramanujan-type identities, most obtained by exploring different initial Bailey pairs and chain parameters.

4. **Modern developments**: Bailey pairs connect to representation theory of affine Lie algebras, vertex operator algebras, and conformal field theory. The Kazhdan-Lusztig correspondence relates Bailey pairs to quantum group representations at roots of unity.

The q-Kangaroo database includes three canonical pairs (`"unit"`, `"rogers-ramanujan"`, `"q-binomial"`) that serve as seeds for generating entire families of identities through the chain and discovery machinery. Together with the weak lemma verification and automated discovery, these tools provide a computational laboratory for exploring the rich structure of $q$-series identities.